### About

This is the notebook for predicting wining side by player performance, namely KDA and minion killed.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import joblib
from pandas import DataFrame as df

In [ ]:
df = pd.read_csv('cleanData.csv')
df.shape

#### Baseline

The below codes tell us percentages of matches are won by team 1. Herem team 1 won less matches than team 2. And if we blindly choose team 2 to be the winner for all matches, this tells us the "accuracy" for this non-sense method and serves as the baseline. 

In [ ]:
t1win = 0
for idx, x in df['win'].iteritems():
    if(x==1.0):
        t1win+=1
print(t1win)
print((1733658-t1win)/1733658)

In [ ]:
cols = df.columns.tolist()
print(cols)

#col concerned:
#kills, death, assists, totminionskilled, duration

In [ ]:
teamRoleList = df['team_role'].unique().tolist()
print(teamRoleList)

In [ ]:
def assignRoleWithNum(x):
    return {
        '1 - MID': 100,
        '2 - MID': 100,
        '1 - JUNGLE': 200,
        '2 - JUNGLE': 200,
        '1 - TOP': 300,
        '2 - TOP': 300,
        '1 - DUO_CARRY': 400,
        '2 - DUO_CARRY': 400,
        '1 - DUO_SUPPORT': 400,
        '2 - DUO_SUPPORT': 400,
        '1 - BOT': 400,
        '2 - BOT': 400
    }[x] 


The function above is to assign roles with values so that we would encode the features into regression.

Value of 1, 10, 100, 1000 is tried.
Turns out that 100 is the best choice (but only 1% accuracy increase, so not much). My educational guess behind this is that kill/death/assists is in range of 0-10. Duration is usually a 4 digits value. Having 100, distinguish this team role feature from others.

In [ ]:
df['role'] = df['team_role'].apply(assignRoleWithNum)
dataset = df[['win','kills','deaths','assists','totminionskilled','duration']]
dataset = dataset.dropna() #drop all nan rows
dataset = dataset.take(np.random.permutation(len(dataset)))#randomize rows
dataset.head(10)

In [ ]:
train, test = train_test_split(dataset, test_size = 0.1)
print('train:', train.shape, 'test:', test.shape)

In [ ]:
def get_data_feed(dataset):
    team_data = dataset.iloc[:,1:] #exclude first column which is win
    winners = dataset['win']
    return team_data, winners 

trainX, trainY = get_data_feed(train)
testX, testY = get_data_feed(test)

In [ ]:
trainX

In [ ]:
LR_Model = LogisticRegression()
LR_Model.fit(trainX, trainY)

prediction = LR_Model.predict(testX)


print ("Accuracy : ", accuracy_score(testY, prediction))

In [ ]:
prediction

In [ ]:
variables = ["kills", "deaths", "assists", "totminionskilled", "duration"]
vals = [[1, 2, 3, 4, 5]]
params = df(vals, columns = variables)
print(params)